I'll first import the mnist_load notebook just like a python module.
This way I don't have to worry about extracting and reshaping
the train and test data again.

In [ ]:
import numpy as np
import sys

In [ ]:
#It has the code to import notebooks;
import ipynb_reader
#my notebook that reads and reshape the data;
import mnist_load as load

train_images_array = load.train_images_array;
train_labels_array = load.train_labels_array;
test_images_array = load.test_images_array;
test_labels_array = load.test_labels_array;

In [ ]:
print(train_images_array.shape);

In [ ]:
#change numpy print threshold;
#set to 'inf' if you wish to print the whole vector/matrix; 
#np.set_printoptions(threshold='1000');

Now I'm going to effectively start the challenge:
Code our previous neural network logic from memory using the two new methods seen in this chapter i.e. Dropout and mini-batched stochastic gradient descent.

In [ ]:
np.random.seed(1);

#rest in peace
relu = lambda x : (x > 0) * x;
relu2deriv = lambda x : (x > 0);

#hidden layer activation function
tanh = lambda input: sinh(input)/cosh(input)
sinh = lambda x: np.exp(x) - np.exp(np.array([i*-1 for j in x for i in j]).reshape(x.shape)) / 2
cosh = lambda x: np.exp(x) + np.exp(np.array([i*-1 for j in x for i in j]).reshape(x.shape)) / 2

tanh2deriv = lambda output: 1 - output ** 2;

#output layer activation function
def softmax(logits):
    tmp = np.exp(logits);
    return tmp / np.sum(tmp, axis=1, keepdims=1);

def softmax2deriv(output, true):
    delta = output - true
    return delta/len(true);

#error function
def cross_entropy(p, q): 
    return - (np.sum(p * np.log(q))/batch_size);

def get_batch_section(layer, row_from, row_to, col_from, col_to):
    sect = layer[:, row_from:row_to, col_from:col_to];
    return sect.reshape((-1 ,1, row_to-row_from, col_to-col_from));
    
one_hot_distribution = np.array([0, 0, 1, 0, 0]);
actual_probability = np.array([.2, .3, .1, .3, .1]);

batch_size = 100;
alpha, iterations = (2, 500);
input_rows, input_cols = 28, 28;
pixels_per_image, labels_per_image = (784, 10);
kernel_rows, kernel_cols, num_of_kernels = (3, 3, 10);

hidden_size = (input_rows - kernel_rows) \
                    * (input_cols - kernel_cols) * num_of_kernels;
    
kernels = .02 * np.random.random((kernel_rows*kernel_cols, num_of_kernels)) - .01;
#synapse_0 = .02 * np.random.random((pixels_per_image, hidden_size)) - .01;
synapse_1 =  .02 * np.random.random((hidden_size, labels_per_image)) - .01;

for iteration in range(iterations):
    #train_msquared_error = .0;
    cross_entropy_train, train_correct_cnt = .0, .0;
    for index in range(int(len(train_images_array)/batch_size)):
        batch_start, batch_end = index * batch_size, \
                                    (index + 1) * batch_size;
        #forward propagation
        layer_0 = train_images_array[batch_start:batch_end];
        
        sects = list();
        for row_start in range(layer_0.shape[1]-kernel_rows):
            for col_start in range(layer_0.shape[2]-kernel_cols):
                sect = get_batch_section(layer_0,
                                         row_start, row_start+kernel_rows,
                                         col_start, col_start+kernel_cols);
                sects.append(sect);
            
        expanded_input = np.concatenate(sects, axis=1);
        es = expanded_input.shape;
        flattened_input = expanded_input.reshape((es[0]*es[1], -1))
        kernels_output = flattened_input.dot(kernels);
        
        layer_1 = np.tanh(kernels_output.reshape((es[0], -1)));

        #dropout mask
        dropout_mask = np.random.randint(2, size=layer_1.shape);
        layer_1 *= dropout_mask * 2;
        
        layer_2 = softmax(layer_1.dot(synapse_1));
        
        #error function
        #train_msquared_error += np.sum((layer_2 - \
        #                        train_labels_array[batch_start:batch_end]) ** 2);  
        cross_entropy_train += \
                    cross_entropy(train_labels_array[batch_start:batch_end], layer_2);
        
        for key in range(batch_size):
            train_correct_cnt += np.argmax(layer_2[key:key+1]) \
                                    == np.argmax( \
                                     train_labels_array[batch_start+key:batch_start+key+1]);
        
        #Global Correlation Summarization
        #Backpropagation or cross-communication
        layer_2_delta = (layer_2 - train_labels_array[batch_start:batch_end]) / (batch_size \
                                  #* softmax2deriv(layer_2, train_labels_array[batch_start:batch_end]);
                                   * layer_2.shape[0]);         
        layer_1_delta = layer_2_delta.dot(synapse_1.T) * tanh2deriv(layer_1);
        
        ##is not necessary giving that layer_1 has the result stored within, after dropout multiplication.
        #layer_1_delta *= dropout_mask * 2;        

        #Local Correlation Summarization
        ##Gradient Descent optimization
        ###Gradient
        synapse_1_delta = layer_1.T.dot(layer_2_delta);
        #before convolutional kernel
        #synapse_0_delta = layer_0.T.dot(layer_1_delta);
        l1d_reshape = layer_1_delta.reshape(kernels_output.shape);
        kernels_delta = flattened_input.T.dot(kernels_output);
        ##Descent
        synapse_1 -= synapse_1_delta * alpha;
        kernels -= kernels_delta * alpha;
    
    #time for inference
    if(iteration % 10 == 9):
        #test_msquared_error = 0;
        cross_entropy_test, test_correct_cnt = 0, 0;
        for index in range(len(test_images_array)):
            layer_0 = test_images_array[index:index+1];
            sects = list();
            for row_start in range(input_rows-kernel_rows):
                for col_start in range(input_cols-kernel_cols):
                    sect = get_batch_section(layer_0,
                                     row_start, row_start+kernel_rows,
                                     col_start, col_start+kernel_cols)
                    sects.append(sect);
            
            expanded_input = np.concatenate(sects, axis=1);
            es = expanded_input.shape;
            flattened_input = expanded_input.reshape((es[0]*es[1], -1));
            kernels_output = flattened_input.dot(kernels);
                       
            layer_1 = tanh(kernels_output.reshape((es[0], -1)));
            layer_2 = softmax(layer_1.dot(synapse_1));
            
            #error function
            #test_msquared_error += np.sum((layer_2 
            #                               - test_labels_array[index:index+1]) ** 2);
            
            cross_entropy_test += cross_entropy(test_labels_array, layer_2);
            test_correct_cnt += np.argmax(layer_2) \
                                        == np.argmax(test_labels_array[index:index+1]);
                                  
        sys.stdout.write('\r' + \
                             'I: ' + str(iteration) + \
                             ' Test_Error: ' + str(cross_entropy_test)[0:5] + \
                             ' Test_Accuracy: ' + str(test_correct_cnt/len(test_images_array)) + \
                             ' Train_Error: ' + str(cross_entropy_train)[0:5] + \
                             ' Train_Accuracy: ' + str(train_correct_cnt/len(train_images_array)));
                             

Trask's implementation

In [ ]:
def tanh(x):
    return np.tanh(x)
def tanh2deriv(output):
    return 1 - (output ** 2)
def softmax(x):
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims=True)

alpha, iterations = (2, 300)
pixels_per_image, num_labels = (784, 10)
batch_size = 128
input_rows = 28
input_cols = 28
kernel_rows = 3
kernel_cols = 3
num_kernels = 16

hidden_size = ((input_rows - kernel_rows) *
 (input_cols - kernel_cols)) * num_kernels
kernels = 0.02*np.random.random((kernel_rows*kernel_cols,
 num_kernels))-0.01
weights_1_2 = 0.2*np.random.random((hidden_size,
 num_labels)) - 0.1
def get_image_section(layer,row_from, row_to, col_from, col_to):
    section = layer[:,row_from:row_to,col_from:col_to]
    return section.reshape(-1,1,row_to-row_from, col_to-col_from)
 
for j in range(iterations):
    correct_cnt = 0
    for i in range(int(len(images) / batch_size)):
        batch_start, batch_end=((i * batch_size),((i+1)*batch_size))
        layer_0 = images[batch_start:batch_end]
        layer_0.shape
        sects = list()
        for row_start in range(layer_0.shape[1]-kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0,
                                         row_start,
                                         row_start+kernel_rows,
                                         col_start,
                                         col_start+kernel_cols)
            sects.append(sect)
        expanded_input = np.concatenate(sects,axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0]*es[1],-1)
 
        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0],-1))
        dropout_mask = np.random.randint(2,size=layer_1.shape)
        layer_1 *= dropout_mask * 2
        layer_2 = softmax(np.dot(layer_1,weights_1_2))
        
        for k in range(batch_size):
            labelset = labels[batch_start+k:batch_start+k+1]
            _inc = int(np.argmax(layer_2[k:k+1]) ==
            np.argmax(labelset))
            correct_cnt += _inc
        
        layer_2_delta = (labels[batch_start:batch_end]-layer_2)\ 
                         / (batch_size * layer_2.shape[0])
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * \ 
                            tanh2deriv(layer_1)
        layer_1_delta *= dropout_mask
        weights_1_2 += alpha * layer_1.T.dot(layer_2_delta)
        l1d_reshape = layer_1_delta.reshape(kernel_output.shape)
        k_update = flattened_input.T.dot(l1d_reshape)
        kernels -= alpha * k_update
        
    test_correct_cnt = 0
    for i in range(len(test_images)):
        layer_0 = test_images[i:i+1]
        layer_0.shape
        
        sects = list()
        for row_start in range(layer_0.shape[1]-kernel_rows):
            for col_start in range(layer_0.shape[2] - kernel_cols):
                sect = get_image_section(layer_0,
                                        row_start,
                                        row_start+kernel_rows,
                                        col_start,
                                        col_start+kernel_cols)
                sects.append(sect)
                
        expanded_input = np.concatenate(sects,axis=1)
        es = expanded_input.shape
        flattened_input = expanded_input.reshape(es[0]*es[1],-1)
        kernel_output = flattened_input.dot(kernels)
        layer_1 = tanh(kernel_output.reshape(es[0],-1))
        layer_2 = np.dot(layer_1,weights_1_2)
        test_correct_cnt += int(np.argmax(layer_2) ==
                                np.argmax(test_labels[i:i+1]))
        if(j % 1 == 0):
 sys.stdout.write("\n"+ \ 
 "I:" + str(j) + \ 
 " Test-Acc:"+str(test_correct_cnt/float(len(test_images)))+\ 
 " Train-Acc:" + str(correct_cnt/float(len(images))))